# PR 4.3
Van Pjotr en Sennen

Hierin gaan wij een paar queries aanvragen aan de database die wij hebben gemaakt gebasseerd op de ETL diagram van de Great_Outdoors.

Hieronder zullen we beginnen met de setup van de libraries en het verbinden met de database. We zullen de pyodbc library gebruiken om de verbinding tussen SSMS en Python. Verder gebruiken we ook pandas om data makkelijk te lezen.

In [1]:
import pandas as pd
import pyodbc
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine
load_dotenv()

True

In [2]:
DB = {'servername': os.getenv('NAME'),
      'database': os.getenv('DATABASE'),
      'username': os.getenv('USER'),
      'password': os.getenv('PASSWORD')}

conn_str = f"DRIVER=SQL Server;SERVER={DB['servername']};DATABASE={DB['database']};UID={DB['username']};PWD={DB['password']};Trusted_Connection=yes;"

conn = pyodbc.connect(conn_str)
export_cursor = conn.cursor()
# Hoe checken we of de connectie werkt?
print(export_cursor.execute("SELECT @@version;"))

Om ervoor te zorgen dat we weten dat we data kunnen aanvragen vanaf de server zullen we hier een paar queries executeren. Zodat we kunnen bevestigen dat we alles goed hebben geconfigureerd. Eerst pakken we alle tabelen van uit de database.

In [3]:
export_cursor.execute("SELECT t.name FROM sys.tables t")
tables = export_cursor.fetchall()

# Voor elke tabel in de database print de naam van de tabel
if(tables == []):
    print("No tables found, the database is empty.")
else:
    for table in tables:
        table = table[0]
        print(table[0])

No tables found, the database is empty.


Daarna wat we willen zijn dus de functies maken die we later in het project gaan gebruiken om de data in te laden in ons project. Zodat we de data uit de bron kunnen transformeren en overzetten naar onze SQL Server database. We maken hiervoor een extract functie maken en een laad functie.

In [23]:
msa_drivers = [x for x in pyodbc.drivers() if 'ACCESS' in x.upper()]
print(f'MS-Access drivers: \n{msa_drivers}')
pyodbc.drivers() 
# pyodbc.connect('Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=H:\HHS\Leerjaar 2\Semester 2\Data Engineering & Data Science\Week 5\Warehouse\go_sales.accdb;')

MS-Access drivers: 
[]


['SQL Server',
 'SQL Server Native Client RDA 11.0',
 'MySQL ODBC 8.0 ANSI Driver',
 'MySQL ODBC 8.0 Unicode Driver',
 'ODBC Driver 17 for SQL Server',
 'ODBC Driver 18 for SQL Server']

## Extract
Eerst gaan we de data uit de access database halen en ervoor zorgen dat we ze later goed kunnen transformeren in de database.

In [25]:
def extract_data_from_access():
    conn_access = pyodbc.connect(r'Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=./go_sales.accdb;')

    product = pd.read_sql("SELECT * FROM product", conn_access)
    product_type = pd.read_sql("SELECT * FROM product_type", conn_access)
    product_line = pd.read_sql("SELECT * FROM product_line", conn_access)
    ProductMerge1 = pd.merge(product, product_type, how="left", left_on="PRODUCT_TYPE_CODE", right_on="PRODUCT_TYPE_CODE")
    ProductExtract = pd.merge(ProductMerge1, product_line, how="left", left_on="PRODUCT_LINE_CODE", right_on="PRODUCT_LINE_CODE")
    ProductExtract

extract_data_from_access()

InterfaceError: ('IM002', '[IM002] [Microsoft][ODBC-stuurprogrammabeheer] De naam van de gegevensbron is niet gevonden en er is geen standaardstuurprogramma opgegeven (0) (SQLDriverConnect)')

## Transform
Nadat we de data eruit hebben gehaald, gaan de data transformeren, zodat we ze in de database kunnen stoppen.

## Loading

Hieronder gaan we de data inladen. Hieronder zal ik de code beschikbaar zijn.